In [1]:
import sys
sys.path.append('../..')
import os
sys.path.append(os.getcwd())
import numpy as np
from minst.core import *

In [ ]:
male_heights = np.random.normal(171, 6, 500)
female_heights = np.random.normal(158, 5, 500)

male_weights = np.random.normal(70, 10, 500)
female_weights = np.random.normal(57, 8, 500)

male_bfrs = np.random.normal(16, 2, 500)
female_bfrs = np.random.normal(22, 2, 500)

male_labels = [1] * 500
female_labels = [-1] * 500

train_set = np.array([np.concatenate((male_heights, female_heights)),
                      np.concatenate((male_weights, female_weights)),
                      np.concatenate((male_bfrs, female_bfrs)),
                      np.concatenate((male_labels, female_labels))]).T
np.random.shuffle(train_set)



In [ ]:
batch_length = len(train_set)
# 构造计算图：输入向量，是一个100x1矩阵，不需要初始化，不参与训练
x =variable_node.Variable(dim=(batch_length, 3), init=False, trainable=False)
x.value = train_set[:, 0:3]
# 类别标签，1男，-1女
label =variable_node.Variable(dim=(batch_length, 1), init=False, trainable=False)
label.set_value(train_set[:, -1])
# print(label.value)
# print(x.value)

# 权重向量，是一个1x3矩阵，需要初始化，参与训练
w =variable_node.Variable(dim=(3, 1), init=True, trainable=True)

# 阈值，是一个1x1矩阵，需要初始化，参与训练
b =variable_node.Variable(dim=(1, 1), init=True, trainable=True)


In [ ]:
# 构造计算图：xw是一个100x1矩阵，表示输入向量与权重向量的乘积，参与训练
xw=operate_node.MatMul(x, w)  
output = operate_node.Add(xw, b)


In [ ]:

predict = activity_node.Step(output)

# 损失函数
#loss = ms.ops.loss.PerceptionLoss(ms.ops.MatMul(label, output))
output.value=output.value.T
loss=loss_node.LogLoss(operate_node.MatMul(label, output))
default_graph = graph.default_graph
loss.forward()
print("loss:", loss.value)


In [5]:
import numpy as np
label=np.array([[1], [0], [1]])
result=np.array([[0.9], [0.5], [0.8]])
# -label*np.log(result)-(1-label)*np.log(1-result)
print (1/(1+np.power(np.e,100)))

3.7200759760208555e-44
